# Data preprocessing and NLP analysis on stock news data

Objectives:

1. Collect historical stock price (2017-2022) for Tesla. Create the trend labels to reflect the movement of the stock on a given day. A cutoff ratio can be given to identify the cases with no stock value change.
2. Use Google’s pre-trained FinBERT tokenizer and model to generate sentiment scores based on news title + news summary. Group the sentiment scores by date to find sentiment scores for each day. (The historical company/stock news data (2017-2022) were collected through running query_google_newsdata.py)
3. Merge daily news sentiment scores and stock price data.

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
from yahoofinancials import YahooFinancials
from tqdm import tqdm
from torch.nn.functional import softmax
import torch
from datetime import datetime, timedelta

## Collect and process stock data from Yahoo Finance

In [3]:
#get stock data from yahoo finance
def stock_dataset(ticker,  start_date="2017-01-01", end_date="2022-01-01", cutoff=1) :
    '''download and process stock data from yahoo finance
       cutoff: cutoff value is set to indicate a percentage under
       that no price change is considered. cutoff=1 means
       increase or decrease 1% price change from close-to-close'''
    stock_df = yf.download(ticker, 
                      start=start_date, 
                      end=end_date, 
                      progress=False,
            )
    #get day of week column
    stock_df['Weekday'] = stock_df.index.day_name()
    stock_df['Price_change'] = np.nan
    stock_df['Tomorrow_price_change'] = np.nan
    dates_index = stock_df.index
    yesterday = str(dates_index[0].date())  
    for date in dates_index[0:]:
        today = str(date.date())
        yesterday = str(date - timedelta(days = 1))
        tomorrow = str(date + timedelta(days = 1))
        #single index condition
        if yesterday in stock_df.index:
            today_close = stock_df.loc[today, 'Close']
            yes_close = stock_df.loc[yesterday, 'Close']
            diff_per = 100*(today_close - yes_close)/yes_close
            if diff_per < -cutoff:
                price_change = 0
                stock_df.loc[today, 'Price_change'] = price_change
            elif diff_per > cutoff:
                price_change = 1
                stock_df.loc[today, 'Price_change'] = price_change
         #single index condition
        if tomorrow in stock_df.index:
            today_close = stock_df.loc[today, 'Close']
            tomo_close = stock_df.loc[tomorrow, 'Close']
            diff_per1 = 100*(tomo_close - today_close)/today_close
            if diff_per1 < -cutoff:
                price_change = 0
                stock_df.loc[today, 'Tomorrow_price_change'] = price_change 
            elif diff_per1 > cutoff:
                price_change = 1
                stock_df.loc[today, 'Tomorrow_price_change'] = price_change 
    return stock_df

In [4]:
stock_df = stock_dataset('TSLA', "2017-01-01", "2022-01-01", 0.5)
stock_df

,Open,High,Low,Close,Adj Close,Volume,Weekday,Price_change,Tomorrow_price_change
Date,,,,,,,,,
2017-01-03,42.972000,44.066002,42.192001,43.397999,43.397999,29616500,Tuesday,NaN,1.0
2017-01-04,42.950001,45.599998,42.862000,45.397999,45.397999,56067500,Wednesday,1.0,NaN
2017-01-05,45.284000,45.495998,44.389999,45.349998,45.349998,29558500,Thursday,NaN,1.0
2017-01-06,45.386002,46.062000,45.090000,45.801998,45.801998,27639500,Friday,1.0,NaN
2017-01-09,45.793999,46.383999,45.599998,46.256001,46.256001,19897500,Monday,NaN,0.0
...,...,...,...,...,...,...,...,...,...
2021-12-27,1073.670044,1117.000000,1070.719971,1093.939941,1093.939941,23715300,Monday,NaN,0.0
2021-12-28,1109.489990,1119.000000,1078.420044,1088.469971,1088.469971,20108000,Tuesday,0.0,NaN
2021-12-29,1098.640015,1104.000000,1064.140015,1086.189941,1086.189941,18718000,Wednesday,NaN,0.0


## Load and process news data from web scraping

In [4]:
#read news data
#newsarticles_tesla.csv is from running webscraping script query_google_newsdata.py
tesla = './newsarticles_tesla.csv'
df_tesla = pd.read_csv(tesla)
df_tesla.head()

,Date,Summary,Title,Article,Link
0,01-01-2017,REUTERS/Patrick T. Fallon The title of this st...,Tesla faces 3 major tests in 2017 — and it has...,Tesla Motors CEO Elon Musk. REUTERS/Patrick T....,https://www.businessinsider.com/tesla-challeng...
1,01-01-2017,With private hire car scheme: -• You set up a ...,90% car loan in Singapore - How it works?,90% Car Loan in Singapore – How does it work?\...,https://finance.yahoo.com/news/90-car-loan-in-...
2,01-01-2017,George intends to buy the stock now and tries ...,Don't Invest Like Jerry Seinfeld And George Co...,(Photo credit)\n\nI don't like to discuss figu...,https://seekingalpha.com/article/4033684-dont-...
3,01-02-2017,"Last week, we reported on a Model X owner suin...",After launching his Model X into his own livin...,"Last week, we reported on a Model X owner suin...",https://electrek.co/2017/01/02/tesla-model-x-s...
4,01-02-2017,"Wow, the internet is really not happy with Joh...","Wow, the internet is really not happy with Joh...","Wow, the internet is really not happy with Joh...",https://finance.yahoo.com/news/wow-internet-re...


In [5]:
df_tesla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7089 entries, 0 to 7088
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     7089 non-null   object
 1   Summary  6707 non-null   object
 2   Title    6744 non-null   object
 3   Article  6707 non-null   object
 4   Link     7089 non-null   object
dtypes: object(5)
memory usage: 277.0+ KB


In [6]:
#there are small portion of null values due to subscription issue. 
#remove rows with null values
df = df_tesla.dropna()
#remove rows with invalid data by check if 'tesla' or 'TSLA' appears in headline or summary
df = df[(df['Summary'].str.lower().str.contains('tesla'))|(df['Summary'].str.lower().str.contains('tsla'))|\
    (df['Title'].str.lower().str.contains('tesla'))|(df['Title'].str.lower().str.contains('tsla'))]
#combine title and article, drop link
df['Title_summary'] = df['Title'] + '.\n' + df['Summary']
df = df.drop(columns=['Link', 'Summary', 'Title', 'Article'], axis=1).set_index(['Date'])
df.reset_index(inplace=True)
df

,Date,Title_summary
0,01-01-2017,Tesla faces 3 major tests in 2017 — and it has...
1,01-02-2017,After launching his Model X into his own livin...
2,01-03-2017,"Tesla misses Q4 delivery mark, but orders are ..."
3,01-04-2017,Tesla (TSLA) rallies +4.5% as Wall Street shru...
4,01-04-2017,"Tesla is surging.\nMarkets InsiderOn Tuesday, ..."
...,...,...
3666,12-30-2021,Why Tesla Stock Dropped Again Today.\nLast nig...
3667,12-30-2021,"Tesla recalls over 475,000 Model S and Model 3..."
3668,12-31-2021,What Next For Tesla Stock?.\nAlthough Tesla st...
3669,12-31-2021,"Pre-Market Movers: AMD rises, Peloton and Tesl..."


In [7]:
df.loc[3666, 'Title_summary']

"Why Tesla Stock Dropped Again Today.\nLast night, Cathie Wood's ARK Invest reported selling 27,900 shares of Tesla, its third straight day of selling, according to data from TheFly.com.\n(Perhaps not coincidentally, this was Tesla's third straight day of stock price declines as well).\nSo whatOn top of ARK's stock sale, Citigroup raised its price target on the stock, but only to $262 per share, a target that (if correct) implies that Tesla stock could fall as much as 75% over the coming year.\nMoreover, Reuters is reporting that 119,009 Tesla Model S sedans of unspecified model years will be recalled because the front hood may open without warning and obstruct the driver's visibility.\nAs consumers digest those numbers and decide whether they want to plunk down tens of thousands of dollars on a new Tesla in 2022, the biggest surprise today might be that the stock price hasn't fallen even further."

## NLP analysis of news data

In [8]:
#initilize pre-trained transformer model finBERT
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def sentim_analyzer_finbert(df, tokenizer, model, column_name):
    ''' apply pre-trained model finBERT to df[column_name] and generate sentiment scores'''
    for i in tqdm(df.index) :
        try:
            col = df.loc[i, column_name].split('.\n')
        except:
            return print(column_name +' column might be missing from dataframe')
        # Pre-process input phrase
        input = tokenizer(col, padding = True, truncation = True, return_tensors='pt')
        # Estimate output
        output = model(**input)
        # Pass model output logits through a softmax layer.
        predictions = softmax(output.logits, dim=-1)
        df.loc[i, 'Positive'] = torch.mean(predictions[:,0]).tolist()
        df.loc[i, 'Negative'] = torch.mean(predictions[:,1]).tolist()
        df.loc[i, 'Neutral']  = torch.mean(predictions[:,2]).tolist()
    return df

#A pre-processing tokenizer object from Hugging Face lib.
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert") 
#A hugging face transformer model
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")


In [9]:
trained_df = sentim_analyzer_finbert(df, tokenizer, model, 'Title_summary')
#save trained_df into csv file
#trained_df.to_csv('trained.csv', index = False)

100%|█████████████████████████████████████████████████████████████████████████████| 3671/3671 [1:03:48<00:00,  1.04s/it]


In [10]:
trained_df = pd.read_csv('trained_tesla.csv')
trained_df

,Date,Title_summary,Positive,Negative,Neutral
0,01-01-2017,Tesla faces 3 major tests in 2017 — and it has...,0.273610,0.070236,0.656154
1,01-02-2017,After launching his Model X into his own livin...,0.035694,0.431789,0.532517
2,01-03-2017,"Tesla misses Q4 delivery mark, but orders are ...",0.550024,0.351256,0.098720
3,01-04-2017,Tesla (TSLA) rallies +4.5% as Wall Street shru...,0.429189,0.107329,0.463483
4,01-04-2017,"Tesla is surging.\nMarkets InsiderOn Tuesday, ...",0.146316,0.513131,0.340553
...,...,...,...,...,...
3666,12-30-2021,Why Tesla Stock Dropped Again Today.\nLast nig...,0.104855,0.694835,0.200310
3667,12-30-2021,"Tesla recalls over 475,000 Model S and Model 3...",0.031609,0.624603,0.343788
3668,12-31-2021,What Next For Tesla Stock?.\nAlthough Tesla st...,0.505123,0.031279,0.463598
3669,12-31-2021,"Pre-Market Movers: AMD rises, Peloton and Tesl...",0.247220,0.679088,0.073692


In [11]:
def news_dataset(df) :
    '''process sentiment data'''
    trained_df['Date'] =pd.to_datetime(trained_df.Date)
    s_df = trained_df[['Date','Positive','Negative','Neutral']].groupby('Date').mean()

    #s_df.set_index(['Date'])
    dates_index = s_df.index
    s_df['Tomorrow_pos'] = np.nan
    s_df['Tomorrow_neg'] = np.nan
    s_df['Tomorrow_neu'] = np.nan
    for date in dates_index[0:]:
        today = str(date.date())
        tomorrow = str((date + timedelta(days = 1)).date())

     #single index condition
        if tomorrow in s_df.index:
            s_df.loc[today, 'Tomorrow_pos'] = s_df.loc[tomorrow, 'Positive']
            s_df.loc[today, 'Tomorrow_neg'] = s_df.loc[tomorrow, 'Negative']
            s_df.loc[today, 'Tomorrow_neu'] = s_df.loc[tomorrow, 'Neutral']
    return s_df.reset_index().sort_values('Date')

sentiment_df = news_dataset(trained_df)
sentiment_df

,Date,Positive,Negative,Neutral,Tomorrow_pos,Tomorrow_neg,Tomorrow_neu
0,2017-01-01,0.273610,0.070236,0.656154,0.035694,0.431789,0.532517
1,2017-01-02,0.035694,0.431789,0.532517,0.550024,0.351256,0.098720
2,2017-01-03,0.550024,0.351256,0.098720,0.394854,0.198030,0.407116
3,2017-01-04,0.394854,0.198030,0.407116,0.353168,0.051893,0.594939
4,2017-01-05,0.353168,0.051893,0.594939,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1605,2021-12-27,0.246671,0.165444,0.587885,0.412727,0.135007,0.452266
1606,2021-12-28,0.412727,0.135007,0.452266,0.208069,0.167552,0.624379
1607,2021-12-29,0.208069,0.167552,0.624379,0.068232,0.659719,0.272049
1608,2021-12-30,0.068232,0.659719,0.272049,0.346728,0.297964,0.355307


## Merge stock data with news sentiment data

In [12]:
def merge_stock_news(df_stock, df_news, how='inner') :
    ''' Merges the financial data dataframe with the news sentiment dataframe'''
    # merge on date column and only for their intersection
    merged_df = df_stock.merge(df_news, on='Date', how=how)
    # rearrange column order
    merged_df = merged_df[['Date', 'Weekday','Positive','Negative','Neutral',\
                           'Tomorrow_pos','Tomorrow_neg','Tomorrow_neu','Price_change','Tomorrow_price_change',\
                           'Open','Close', 'Volume','High','Low', 'Adj Close']]
    merged_df['Date'] =pd.to_datetime(merged_df['Date'])
    #shift the data by one more day and rename all the columns
    merged_df['Date'] = merged_df['Date'] + pd.Timedelta('1 day')
    merged_df['Weekday'] = merged_df['Date'].dt.day_name()
    merged_df = merged_df.rename(columns={'Positive': 'Yesterday_pos', 'Negative': 'Yesterday_neg', 'Neutral': 'Yesterday_neu',\
                           'Tomorrow_pos':'Positive','Tomorrow_neg':'Negative', 'Tomorrow_neu':'Neutral',\
                           'Price_change':'Yesterday_price_change','Tomorrow_price_change':'Price_change',\
                           'Open':'Yesterday_open','Close':'Yesterday_close', 'Volume':'Yesterday_volume',\
                            'High':'Yesterday_high','Low':'Yesterday_low', 'Adj Close':'Yesterday_Adj_close'})
    return merged_df
merged_df = merge_stock_news(stock_df, sentiment_df)
merged_df

,Date,Weekday,Yesterday_pos,Yesterday_neg,Yesterday_neu,Positive,Negative,Neutral,Yesterday_price_change,Price_change,Yesterday_open,Yesterday_close,Yesterday_volume,Yesterday_high,Yesterday_low,Yesterday_Adj_close
0,2017-01-04,Wednesday,0.550024,0.351256,0.098720,0.394854,0.198030,0.407116,NaN,1.0,42.972000,43.397999,29616500,44.066002,42.192001,43.397999
1,2017-01-05,Thursday,0.394854,0.198030,0.407116,0.353168,0.051893,0.594939,1.0,NaN,42.950001,45.397999,56067500,45.599998,42.862000,45.397999
2,2017-01-06,Friday,0.353168,0.051893,0.594939,NaN,NaN,NaN,NaN,1.0,45.284000,45.349998,29558500,45.495998,44.389999,45.349998
3,2017-01-10,Tuesday,0.029573,0.342362,0.628065,0.131342,0.027569,0.841089,NaN,0.0,45.793999,46.256001,19897500,46.383999,45.599998,46.256001
4,2017-01-11,Wednesday,0.131342,0.027569,0.841089,0.381654,0.172998,0.445348,0.0,NaN,46.400002,45.973999,18300000,46.400002,45.377998,45.973999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,2021-12-28,Tuesday,0.246671,0.165444,0.587885,0.412727,0.135007,0.452266,NaN,0.0,1073.670044,1093.939941,23715300,1117.000000,1070.719971,1093.939941
1150,2021-12-29,Wednesday,0.412727,0.135007,0.452266,0.208069,0.167552,0.624379,0.0,NaN,1109.489990,1088.469971,20108000,1119.000000,1078.420044,1088.469971
1151,2021-12-30,Thursday,0.208069,0.167552,0.624379,0.068232,0.659719,0.272049,NaN,0.0,1098.640015,1086.189941,18718000,1104.000000,1064.140015,1086.189941
1152,2021-12-31,Friday,0.068232,0.659719,0.272049,0.346728,0.297964,0.355307,0.0,0.0,1061.329956,1070.339966,15680300,1095.550049,1053.150024,1070.339966


In [13]:
#save merge_stock_news into csv file
merged_df.to_csv('merge_stock_news_tesla.csv', index = False)